<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


 # Chapter 2 Exercise solutions
 # 第二章 练习解答

 Packages that are being used in this notebook:
 本节中使用的包:

In [1]:
# 导入importlib.metadata模块中的version函数用于获取包版本信息
from importlib.metadata import version

# 打印PyTorch包的版本号
print("torch version:", version("torch"))
# 打印tiktoken包的版本号
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.0
tiktoken version: 0.8.0


# Exercise 2.1

In [15]:
# 导入tiktoken库用于分词
import tiktoken

# 获取GPT-2的分词器
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
# 使用tokenizer将字符串"Akwirw ier"编码为整数序列
integers = tokenizer.encode("Akwirw ier")
# 打印编码后的整数序列
print(integers)

[33901, 86, 343, 86, 220, 959]


In [4]:
# 遍历整数序列
for i in integers:
    # 打印每个整数及其对应的解码后的字符串
    print(f"{i} -> {tokenizer.decode([i])}")

33901 -> Ak
86 -> w
343 -> ir
86 -> w
220 ->  
959 -> ier


In [5]:
# 使用tokenizer将字符串"Ak"编码为整数序列
tokenizer.encode("Ak")

[33901]

In [6]:
# 使用tokenizer将字符串"w"编码为整数序列
tokenizer.encode("w")

[86]

In [7]:
# 使用tokenizer将字符串"ir"编码为整数序列
tokenizer.encode("ir")

[343]

In [8]:
# 使用tokenizer将字符串"w"编码为整数序列
tokenizer.encode("w")

[86]

In [9]:
# 使用tokenizer将空格字符编码为整数序列
tokenizer.encode(" ")

[220]

In [10]:
# 使用tokenizer将字符串"ier"编码为整数序列
tokenizer.encode("ier")

[959]

In [11]:
# 使用tokenizer将整数序列[33901, 86, 343, 86, 220, 959]解码为字符串
# 这个序列对应的字符串是"Akwirw ier"
tokenizer.decode([33901, 86, 343, 86, 220, 959])

'Akwirw ier'

# Exercise 2.2

In [12]:
# 导入所需的库
import tiktoken  # 用于文本标记化
import torch  # PyTorch深度学习库
from torch.utils.data import Dataset, DataLoader  # 用于创建数据集和数据加载器


# 定义GPT数据集类,继承自Dataset
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # 初始化输入ID和目标ID列表
        self.input_ids = []
        self.target_ids = []

        # 使用tokenizer对整个文本进行标记化
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 使用滑动窗口将文本分成重叠的序列
        for i in range(0, len(token_ids) - max_length, stride):
            # 获取输入序列
            input_chunk = token_ids[i:i + max_length]
            # 获取目标序列(向后移动一位)
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # 将序列转换为tensor并添加到列表中
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # 返回数据集长度
    def __len__(self):
        return len(self.input_ids)

    # 返回指定索引的数据项
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


# 创建数据加载器的函数
def create_dataloader(txt, batch_size=4, max_length=256, stride=128):
    # 初始化GPT-2 tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集实例
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建并返回数据加载器
    dataloader = DataLoader(dataset, batch_size=batch_size)

    return dataloader


# 读取文本文件
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 初始化tokenizer并编码文本
tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

# 设置模型参数
vocab_size = 50257  # 词汇表大小
output_dim = 256    # 输出维度
max_len = 4         # 最大长度
context_length = max_len  # 上下文长度

# 创建词元嵌入层和位置嵌入层
token_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [13]:
# 使用指定参数创建数据加载器(batch_size=4, max_length=2, stride=2)
dataloader = create_dataloader(raw_text, batch_size=4, max_length=2, stride=2)

# 遍历数据加载器获取一个批次的数据
for batch in dataloader:
    # 将批次数据解包为输入x和目标y
    x, y = batch
    # 获取一个批次后跳出循环
    break

x

tensor([[  40,  367],
        [2885, 1464],
        [1807, 3619],
        [ 402,  271]])

In [16]:
# 使用指定参数创建数据加载器(batch_size=4, max_length=8, stride=2)
dataloader = create_dataloader(raw_text, batch_size=4, max_length=8, stride=2)

# 遍历数据加载器获取一个批次的数据
for batch in dataloader:
    # 将批次数据解包为输入x和目标y
    x, y = batch
    # 获取一个批次后跳出循环
    break

# 返回输入数据x
x

tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]])